(ex-electronic-medical-record)=
# (EX) Electronic medical record
This example utilizes the [MIMIC III demo](https://physionet.org/content/mimiciii-demo/1.4/) dataset in illustrating the use of database using DuckDB.

*Quick notes about DuckDB:*  

- DuckDB is a relational database for analytics processing (i.e., OLAP)
- DuckDB is columnar-oriented
- DuckDB scales reasonably for *relatively* large datasets and works well for local development

In [1]:
import duckdb

In [3]:
# establish connection
conn = duckdb.connect('../lab/dataset/mimic.db', read_only=True)

conn.sql('SHOW TABLES;')

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [7]:
# metadata for add
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns 
    WHERE table_name = 'ADMISSIONS';
    """
)

┌──────────────────────┬───────────┐
│     column_name      │ data_type │
│       varchar        │  varchar  │
├──────────────────────┼───────────┤
│ row_id               │ INTEGER   │
│ subject_id           │ INTEGER   │
│ hadm_id              │ INTEGER   │
│ admittime            │ VARCHAR   │
│ dischtime            │ VARCHAR   │
│ deathtime            │ VARCHAR   │
│ admission_type       │ VARCHAR   │
│ admission_location   │ VARCHAR   │
│ discharge_location   │ VARCHAR   │
│ insurance            │ VARCHAR   │
│ language             │ VARCHAR   │
│ religion             │ VARCHAR   │
│ marital_status       │ VARCHAR   │
│ ethnicity            │ VARCHAR   │
│ edregtime            │ VARCHAR   │
│ edouttime            │ VARCHAR   │
│ diagnosis            │ VARCHAR   │
│ hospital_expire_flag │ INTEGER   │
│ has_chartevents_data │ INTEGER   │
├──────────────────────┴───────────┤
│ 19 rows                2 columns │
└──────────────────────────────────┘

In [8]:
# admission table at a glance
conn.sql(
    """
    SELECT * FROM ADMISSIONS;
    """
)

┌────────┬────────────┬─────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ row_id │ subject_id │ hadm_id │ … │      diagnosis       │ hospital_expire_flag │ has_chartevents_data │
│ int32  │   int32    │  int32  │   │       varchar        │        int32         │        int32         │
├────────┼────────────┼─────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│  12258 │      10006 │  142345 │ … │ SEPSIS               │                    0 │                    1 │
│  12263 │      10011 │  105331 │ … │ HEPATITIS B          │                    1 │                    1 │
│  12265 │      10013 │  165520 │ … │ SEPSIS               │                    1 │                    1 │
│  12269 │      10017 │  199207 │ … │ HUMERAL FRACTURE     │                    0 │                    1 │
│  12270 │      10019 │  177759 │ … │ ALCOHOLIC HEPATITIS  │                    1 │                    1 │
│  12277 │      10026 │  103770 │ … │

## Simple queries

In [9]:
# number of patients
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS;
    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          100 │
└──────────────┘

In [11]:
# number of patients by gender
conn.sql(
    """
    SELECT COUNT(*), gender FROM PATIENTS
    GROUP BY gender;
    """
)

┌──────────────┬─────────┐
│ count_star() │ gender  │
│    int64     │ varchar │
├──────────────┼─────────┤
│           55 │ F       │
│           45 │ M       │
└──────────────┴─────────┘

In [ ]:
# take a glance at the ICUSTAY table
conn.sql(
    """

    """
)

## Derivative queries

In [15]:
# calculate the age of each patient at the time of admission
# cast(patients.dob as date) allows for addition and subtraction of dates
conn.sql(
    """
    SELECT patients.subject_id, round((cast(admissions.admittime as date) - 
        cast(patients.dob as date))/365) as age1, 
    FROM patients
    INNER JOIN admissions ON patients.subject_id = admissions.subject_id;
    """
)

┌────────────┬────────┐
│ subject_id │  age1  │
│   int32    │ double │
├────────────┼────────┤
│      10006 │   71.0 │
│      10011 │   36.0 │
│      10013 │   87.0 │
│      10017 │   74.0 │
│      10019 │   49.0 │
│      10026 │  300.0 │
│      10027 │   83.0 │
│      10029 │   79.0 │
│      10032 │   88.0 │
│      10033 │   82.0 │
│        ·   │     ·  │
│        ·   │     ·  │
│        ·   │     ·  │
│      43881 │   54.0 │
│      43909 │   79.0 │
│      43927 │   72.0 │
│      44083 │   55.0 │
│      44083 │   55.0 │
│      44083 │   55.0 │
│      44154 │  300.0 │
│      44212 │   45.0 │
│      44222 │   73.0 │
│      44228 │   58.0 │
├────────────┴────────┤
│ 129 rows (20 shown) │
└─────────────────────┘

In [ ]:
# calculate the duration of admission before each patient is admitted to the ICU
conn.sql(
    """
    
    """
)

In [ ]:
# identify if a patient passes away when they are in the ICU
conn.sql(
    """
    
    """
)

## Additional reference queries

In [56]:
# 
conn.sql(
    """
    SELECT p.subject_id, p.dob, a.hadm_id,
        a.admittime, p.expire_flag
    FROM admissions a
    INNER JOIN patients p
    ON p.subject_id = a.subject_id
    ORDER BY admittime;
    """
)

In [16]:
 # 
conn.sql(
    """
    SELECT p.subject_id, p.dob, a.hadm_id,
        a.admittime, p.expire_flag,
        MIN (a.admittime) OVER (PARTITION BY p.subject_id) AS first_admittime
    FROM admissions a
    INNER JOIN patients p
    ON p.subject_id = a.subject_id
    ORDER BY a.hadm_id, p.subject_id;
    """
)

┌────────────┬─────────────────────┬─────────┬─────────────────────┬─────────────┬─────────────────────┐
│ subject_id │         dob         │ hadm_id │      admittime      │ expire_flag │   first_admittime   │
│   int32    │       varchar       │  int32  │       varchar       │    int32    │       varchar       │
├────────────┼─────────────────────┼─────────┼─────────────────────┼─────────────┼─────────────────────┤
│      10056 │ 2046-02-27 00:00:00 │  100375 │ 2129-05-02 00:12:00 │           1 │ 2129-05-02 00:12:00 │
│      42430 │ 2061-12-10 00:00:00 │  100969 │ 2142-11-26 21:20:00 │           1 │ 2142-11-26 21:20:00 │
│      41914 │ 2090-11-16 00:00:00 │  101361 │ 2145-12-01 18:13:00 │           1 │ 2145-12-01 18:13:00 │
│      42135 │ 2083-09-20 00:00:00 │  102203 │ 2127-07-23 15:21:00 │           1 │ 2127-07-23 15:21:00 │
│      44228 │ 2112-10-22 00:00:00 │  103379 │ 2170-12-15 03:14:00 │           1 │ 2170-12-15 03:14:00 │
│      10026 │ 1895-05-17 00:00:00 │  103770 │ 2195-05-